In [20]:
from pyspark.sql.functions import split,lit,rtrim,col,trim

In [3]:
basedos_header = spark.read.text('hdfs:///data/HEADERS/BASEDOS_HEADER.csv').collect()[0][0].split(';')

In [4]:
print(basedos_header)

['TDPRNFOY', 'TDPRNCONT', 'TDPRNCARTE', 'TDPRNDT', 'TDPRMMENS', 'TDPRMSREC', 'TDPRMCRD']


In [5]:
file_date='201801'

In [6]:
basedos_0 = (spark.read.load('hdfs:///data/BASEDOS/BASEDOS.'+file_date+'.csv.bz2',
                                       format ='com.databricks.spark.csv',
                                       header='false',
                                       quote ='\\',
                                       delimiter = ';',
                                       inferSchema='true'))

In [8]:
basedos_2=basedos_0
for ncol in range(len(basedos_header)):
    basedos_2=basedos_2.withColumn(basedos_header[ncol],trim(basedos_2[basedos_2.columns[ncol]])).drop(basedos_0.columns[ncol])

In [9]:
basedos_3=basedos_2.withColumn('DATE_CHARGEMENT',lit(file_date))

In [10]:
basedos_3.show(5)

+----------+---------+----------+-------+---------+---------+--------+---------------+
|  TDPRNFOY|TDPRNCONT|TDPRNCARTE|TDPRNDT|TDPRMMENS|TDPRMSREC|TDPRMCRD|DATE_CHARGEMENT|
+----------+---------+----------+-------+---------+---------+--------+---------------+
|4100000238|        1|     11000| 284049|        1|   284049|  284049|         201801|
|4100000345|        1|      8286|      0|        1|        0|       0|         201801|
|4100000386|        0|     27662|      0|        0|        0|       0|         201801|
|4100000550|        1|      2333|      0|        1|        0|       0|         201801|
|4100001103|        1|      9500| 400000|        1|   400000|  400000|         201801|
+----------+---------+----------+-------+---------+---------+--------+---------------+
only showing top 5 rows



In [12]:
basedos_3.dtypes

[('TDPRNFOY', 'string'),
 ('TDPRNCONT', 'string'),
 ('TDPRNCARTE', 'string'),
 ('TDPRNDT', 'string'),
 ('TDPRMMENS', 'string'),
 ('TDPRMSREC', 'string'),
 ('TDPRMCRD', 'string'),
 ('DATE_CHARGEMENT', 'string')]

In [13]:
requete="""
create external table if not exists default.basedos
(
TDPRNFOY string,
TDPRNCONT int,
TDPRNCARTE int,
TDPRNDT int,
TDPRMMENS int,
TDPRMSREC int,
TDPRMCRD int
)
partitioned by (DATE_CHARGEMENT int)
stored as parquet

location 'hdfs:///tmp/BASEDOS'
"""
spark.sql(requete)

DataFrame[]

In [23]:
def load_basedos(file_date):
    basedos_batch = (spark.read.load('hdfs:///data/BASEDOS/BASEDOS.'+file_date+'.csv.bz2',
                                       format ='com.databricks.spark.csv',
                                       header='false',
                                       quote ='\\',
                                       delimiter = ';',
                                       inferSchema='true'))      
    oldCols=basedos_batch.columns
    for ncol in range(len(basedos_header)):
        basedos_batch=basedos_batch.withColumn(basedos_header[ncol],trim(basedos_batch[oldCols[ncol]])).drop(oldCols[ncol])

    basedos_batch.createOrReplaceTempView('tmpTable')

    requete="""
    insert overwrite table default.basedos partition (DATE_CHARGEMENT=""" + file_date +""")
    (
    select *
    from tmpTable
    )
    """
    spark.sql(requete) 
    print('%s done'%file_date)

In [17]:
import os
res=os.popen('hdfs dfs -ls hdfs:///data/BASEDOS/BASEDOS*').read()

In [18]:
import re
res2 = re.findall('BASEDOS\.([0-9]+)',res)

In [21]:
for file_date in res2:
    load_basedos(file_date)

201712 done
201801 done
201803 done
201804 done
201805 done


In [ ]:
spark.sql("analyze table default.basedos compute statistics noscan")

In [ ]:
%%bash
hdfs dfs -ls /tmp